<a href="https://colab.research.google.com/github/evbevz/epoxy-supervisor/blob/main/EpoxyLevel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
try:
    import google.colab
    IS_COLAB_ENV = True
except:
    IS_COLAB_ENV = False
if IS_COLAB_ENV:
    %pip install ultralytics
    !git clone https://github.com/evbevz/epoxy-supervisor.git
    dirBase = '/content/'
else:
    dirBase = '/home/nikolay/opencv/'
%cd "{dirBase}/epoxy-supervisor/"

In [2]:
import epoxylib
from ultralytics import YOLO

# Значение уверенности в правильности распознавания, ниже которого не будем считать, что точки определились правильно. Т.е. координаты такой точки будем считать ложными и точку игнорировать.
kptConfidence = 0.8 # Сейчас точки если и распознаются, то с уверенностью больше 0.9

# Калибровочный файл в котором хранятся уровни от 0 до 20мл. В виде координат X,Y центра эллипса поверхности эпоксидки на каждом уровне.
filenameAnnotation = dirBase + 'epoxy-supervisor/samples/EpoxyLevelCalibrate.annotations.xml'

# Файл весов обученной для распознавания модели
#filenameAIModel = dirBase + 'epoxy-supervisor/weights/epoxy-supervisor.20241228.best.pt'
filenameAIModel = dirBase + 'epoxy-supervisor/weights/es.yolo11s-pose.20250121.epoch400.pt'

# Изображение для предсказания уровня эпоксидки
#filenameInjectorCam = 'epoxy-supervisor/samples/fail01.png'
#filenameInjectorCam = 'epoxy-supervisor/samples/000030.png'
filenameInjectorCam20 = dirBase + 'epoxy-supervisor/samples/20ml.png'
filenameInjectorCam18 = dirBase + 'epoxy-supervisor/samples/18ml.png'
filenameInjectorCam10 = dirBase + 'epoxy-supervisor/samples/10ml.png'
filenameInjectorCam0 = dirBase + 'epoxy-supervisor/samples/fail01.png'
filenameInjectorCam = dirBase + 'epoxy-supervisor/frames-009/00900040.png'

# Калибруем шприц
arrayEpoxyLevel = epoxylib.Calibrate(filenameAnnotation)

# Загружаем модель
model = YOLO(filenameAIModel)

# Запрос уровня по изображению
level = None
level = epoxylib.GetEpoxyLevel(model, arrayEpoxyLevel, filenameInjectorCam20, kptConfidence, level)
print (f"Уровень эпоксидки: {level}")
level = epoxylib.GetEpoxyLevel(model, arrayEpoxyLevel, filenameInjectorCam18, kptConfidence, level)
print (f"Уровень эпоксидки: {level}")
level = epoxylib.GetEpoxyLevel(model, arrayEpoxyLevel, filenameInjectorCam0, kptConfidence, level)
print (f"Уровень эпоксидки: {level}")
level = epoxylib.GetEpoxyLevel(model, arrayEpoxyLevel, filenameInjectorCam10, kptConfidence, level)
print (f"Уровень эпоксидки: {level}")
level = epoxylib.GetEpoxyLevel(model, arrayEpoxyLevel, filenameInjectorCam, kptConfidence, level)
print (f"Уровень эпоксидки: {level}")


Уровень эпоксидки: 20
Уровень эпоксидки: 18.825317576527596
Уровень эпоксидки: 18.825317576527596
Уровень эпоксидки: 9.97814641147852
Уровень эпоксидки: 10.517840027809143
